In [ ]:
import cv2 as cv
import dlib
import numpy as np
from imutils import face_utils

# Load dlib's face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Ensure this file exists!

# Open camera
cap = cv.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera not accessible!")
    exit()

sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

# Function to compute Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])  # Vertical distance
    B = np.linalg.norm(eye[2] - eye[4])  # Vertical distance
    C = np.linalg.norm(eye[0] - eye[3])  # Horizontal distance
    ear = (A + B) / (2.0 * C)  # EAR formula
    return ear

# Default EAR threshold (will adjust dynamically)
EAR_THRESHOLD = 0.2  # Adjust based on debug output

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Failed to capture frame. Check camera connection.")
        continue  # Skip iteration if frame is not captured

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        left_eye = landmarks[36:42]  # Left eye landmark points
        right_eye = landmarks[42:48]  # Right eye landmark points

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0  # Average EAR

        # 🛠️ **DEBUG: Print EAR values to adjust threshold**
        print(f"Left EAR: {left_ear:.2f}, Right EAR: {right_ear:.2f}, Avg EAR: {avg_ear:.2f}")

        # Show EAR value on the screen
        cv.putText(frame, f"EAR: {avg_ear:.2f}", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Check if the eyes are closed
        if avg_ear < EAR_THRESHOLD:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:  # Eyes closed for 6+ frames
                status = "SLEEPING!"
                color = (0, 0, 255)  # <-- Red color for "SLEEPING!"

        
        elif avg_ear >= EAR_THRESHOLD:
            sleep = 0
            active += 1
            drowsy = 0
            if active > 6:
                status = "ACTIVE!"
                color = (0, 255, 0)

        # Display status on the screen
        cv.putText(frame, status, (100, 100), cv.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        # Draw eye landmarks
        for (x, y) in landmarks[36:48]:  # Only draw eye points
            cv.circle(frame, (x, y), 1, (255, 255, 255), -1)

    cv.imshow("Drowsiness Detector", frame)

    key = cv.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

cap.release()
cv.destroyAllWindows()


Left EAR: 0.33, Right EAR: 0.38, Avg EAR: 0.36
Left EAR: 0.19, Right EAR: 0.22, Avg EAR: 0.20
Left EAR: 0.25, Right EAR: 0.27, Avg EAR: 0.26
Left EAR: 0.26, Right EAR: 0.31, Avg EAR: 0.28
Left EAR: 0.25, Right EAR: 0.28, Avg EAR: 0.26
Left EAR: 0.26, Right EAR: 0.29, Avg EAR: 0.28


2025-03-31 15:22:21.533 python[51018:4837088] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-31 15:22:21.533 python[51018:4837088] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Left EAR: 0.26, Right EAR: 0.30, Avg EAR: 0.28
Left EAR: 0.28, Right EAR: 0.31, Avg EAR: 0.29
Left EAR: 0.31, Right EAR: 0.32, Avg EAR: 0.32
Left EAR: 0.30, Right EAR: 0.33, Avg EAR: 0.31
Left EAR: 0.34, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.35, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.34, Right EAR: 0.36, Avg EAR: 0.35
Left EAR: 0.34, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.33, Right EAR: 0.36, Avg EAR: 0.35
Left EAR: 0.33, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.35, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.34, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.34
Left EAR: 0.33, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.34, Right EAR: 0.36, Avg EAR: 0.35
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.34
Left EAR: 0.35, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.24, Right EAR: 0.25, Avg EAR: 0.24
Left EAR: 0.19, Right EAR: 0.21, Avg EAR: 0.20
Left EAR: 0.17, Right EAR: 0.20, Avg EAR: 0.18
Left EAR: 0.2

Left EAR: 0.20, Right EAR: 0.21, Avg EAR: 0.20
Left EAR: 0.18, Right EAR: 0.20, Avg EAR: 0.19
Left EAR: 0.16, Right EAR: 0.19, Avg EAR: 0.17
Left EAR: 0.25, Right EAR: 0.25, Avg EAR: 0.25
Left EAR: 0.31, Right EAR: 0.32, Avg EAR: 0.32
Left EAR: 0.32, Right EAR: 0.31, Avg EAR: 0.32
Left EAR: 0.33, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.34, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.31, Right EAR: 0.32, Avg EAR: 0.31
Left EAR: 0.35, Right EAR: 0.35, Avg EAR: 0.35
Left EAR: 0.31, Right EAR: 0.31, Avg EAR: 0.31
Left EAR: 0.23, Right EAR: 0.26, Avg EAR: 0.24
Left EAR: 0.19, Right EAR: 0.19, Avg EAR: 0.19
Left EAR: 0.21, Right EAR: 0.23, Avg EAR: 0.22
Left EAR: 0.20, Right EAR: 0.19, Avg EAR: 0.20
Left EAR: 0.20, Right EAR: 0.22, Avg EAR: 0.21
Left EAR: 0.21, Right EAR: 0.24, Avg EAR: 0.23
Left EAR: 0.21, Right EAR: 0.24, Avg EAR: 0.23
Left EAR: 0.2

Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.31, Right EAR: 0.32, Avg EAR: 0.31
Left EAR: 0.33, Right EAR: 0.33, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.34
Left EAR: 0.31, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.33, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.28, Right EAR: 0.30, Avg EAR: 0.29
Left EAR: 0.30, Right EAR: 0.32, Avg EAR: 0.31
Left EAR: 0.31, Right EAR: 0.33, Avg EAR: 0.32
Left EAR: 0.28, Right EAR: 0.31, Avg EAR: 0.30
Left EAR: 0.30, Right EAR: 0.34, Avg EAR: 0.32
Left EAR: 0.30, Right EAR: 0.32, Avg EAR: 0.31
Left EAR: 0.30, Right EAR: 0.34, Avg EAR: 0.32
Left EAR: 0.31, Right EAR: 0.33, Avg EAR: 0.32
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.24, Right EAR: 0.26, Avg EAR: 0.25
Left EAR: 0.24, Right EAR: 0.25, Avg EAR: 0.24
Left EAR: 0.30, Right EAR: 0.33, Avg EAR: 0.31
Left EAR: 0.3

...

Left EAR: 0.26, Right EAR: 0.27, Avg EAR: 0.27
Left EAR: 0.28, Right EAR: 0.29, Avg EAR: 0.29
Left EAR: 0.28, Right EAR: 0.28, Avg EAR: 0.28
Left EAR: 0.28, Right EAR: 0.30, Avg EAR: 0.29
Left EAR: 0.30, Right EAR: 0.31, Avg EAR: 0.30
Left EAR: 0.30, Right EAR: 0.32, Avg EAR: 0.31
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.33, Avg EAR: 0.32
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.34, Right EAR: 0.33, Avg EAR: 0.34
Left EAR: 0.34, Right EAR: 0.35, Avg EAR: 0.34
Left EAR: 0.31, Right EAR: 0.31, Avg EAR: 0.31
Left EAR: 0.17, Right EAR: 0.18, Avg EAR: 0.18
Left EAR: 0.25, Right EAR: 0.25, Avg EAR: 0.25
Left EAR: 0.24, Right EAR: 0.24, Avg EAR: 0.24
Left EAR: 0.20, Right EAR: 0.23, Avg EAR: 0.21
Left EAR: 0.20, Right EAR: 0.26, Avg EAR: 0.23
Left EAR: 0.23, Right EAR: 0.28, Avg EAR: 0.26
Left EAR: 0.23, Right EAR: 0.29, Avg EAR: 0.26
Left EAR: 0.26, Right EAR: 0.32, Avg EAR: 0.29
Left EAR: 0.18, Right EAR: 0.23, Avg EAR: 0.21
Left EAR: 0.2

Left EAR: 0.25, Right EAR: 0.28, Avg EAR: 0.26
Left EAR: 0.24, Right EAR: 0.30, Avg EAR: 0.27
Left EAR: 0.24, Right EAR: 0.30, Avg EAR: 0.27
Left EAR: 0.23, Right EAR: 0.29, Avg EAR: 0.26
Left EAR: 0.18, Right EAR: 0.20, Avg EAR: 0.19
Left EAR: 0.19, Right EAR: 0.21, Avg EAR: 0.20
Left EAR: 0.27, Right EAR: 0.30, Avg EAR: 0.28
Left EAR: 0.27, Right EAR: 0.26, Avg EAR: 0.26
Left EAR: 0.31, Right EAR: 0.29, Avg EAR: 0.30
Left EAR: 0.32, Right EAR: 0.31, Avg EAR: 0.32
Left EAR: 0.31, Right EAR: 0.29, Avg EAR: 0.30
Left EAR: 0.32, Right EAR: 0.29, Avg EAR: 0.30
Left EAR: 0.33, Right EAR: 0.29, Avg EAR: 0.31
Left EAR: 0.32, Right EAR: 0.29, Avg EAR: 0.30
Left EAR: 0.33, Right EAR: 0.30, Avg EAR: 0.31
Left EAR: 0.32, Right EAR: 0.31, Avg EAR: 0.32
Left EAR: 0.32, Right EAR: 0.32, Avg EAR: 0.32
Left EAR: 0.29, Right EAR: 0.29, Avg EAR: 0.29
Left EAR: 0.32, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.33, Right EAR: 0.36, Avg EAR: 0.35
Left EAR: 0.29, Right EAR: 0.33, Avg EAR: 0.31
Left EAR: 0.2

Left EAR: 0.20, Right EAR: 0.20, Avg EAR: 0.20
Left EAR: 0.18, Right EAR: 0.19, Avg EAR: 0.19
Left EAR: 0.19, Right EAR: 0.21, Avg EAR: 0.20
Left EAR: 0.20, Right EAR: 0.24, Avg EAR: 0.22
Left EAR: 0.20, Right EAR: 0.23, Avg EAR: 0.21
Left EAR: 0.27, Right EAR: 0.31, Avg EAR: 0.29
Left EAR: 0.28, Right EAR: 0.32, Avg EAR: 0.30
Left EAR: 0.29, Right EAR: 0.31, Avg EAR: 0.30
Left EAR: 0.31, Right EAR: 0.31, Avg EAR: 0.31
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.33
Left EAR: 0.32, Right EAR: 0.32, Avg EAR: 0.32
Left EAR: 0.33, Right EAR: 0.34, Avg EAR: 0.34
Left EAR: 0.22, Right EAR: 0.24, Avg EAR: 0.23
Left EAR: 0.23, Right EAR: 0.27, Avg EAR: 0.25
Left EAR: 0.21, Right EAR: 0.22, Avg EAR: 0.22
Left EAR: 0.22, Right EAR: 0.23, Avg EAR: 0.23
Left EAR: 0.21, Right EAR: 0.23, Avg EAR: 0.22
Left EAR: 0.20, Right EAR: 0.22, Avg EAR: 0.21
Left EAR: 0.23, Right EAR: 0.22, Avg EAR: 0.23
Left EAR: 0.21, Right EAR: 0.21, Avg EAR: 0.21
Left EAR: 0.23, Right EAR: 0.23, Avg EAR: 0.23
Left EAR: 0.2

Left EAR: 0.29, Right EAR: 0.31, Avg EAR: 0.30
Left EAR: 0.28, Right EAR: 0.31, Avg EAR: 0.30
Left EAR: 0.27, Right EAR: 0.30, Avg EAR: 0.29
Left EAR: 0.27, Right EAR: 0.29, Avg EAR: 0.28
Left EAR: 0.28, Right EAR: 0.30, Avg EAR: 0.29
Left EAR: 0.28, Right EAR: 0.28, Avg EAR: 0.28
Left EAR: 0.29, Right EAR: 0.28, Avg EAR: 0.28
Left EAR: 0.27, Right EAR: 0.26, Avg EAR: 0.26
Left EAR: 0.29, Right EAR: 0.27, Avg EAR: 0.28
Left EAR: 0.29, Right EAR: 0.29, Avg EAR: 0.29
Left EAR: 0.27, Right EAR: 0.27, Avg EAR: 0.27
Left EAR: 0.27, Right EAR: 0.27, Avg EAR: 0.27
Left EAR: 0.25, Right EAR: 0.27, Avg EAR: 0.26
Left EAR: 0.27, Right EAR: 0.28, Avg EAR: 0.28
Left EAR: 0.26, Right EAR: 0.27, Avg EAR: 0.26
Left EAR: 0.27, Right EAR: 0.28, Avg EAR: 0.28
Left EAR: 0.26, Right EAR: 0.28, Avg EAR: 0.27
Left EAR: 0.27, Right EAR: 0.28, Avg EAR: 0.27
Left EAR: 0.28, Right EAR: 0.29, Avg EAR: 0.29
Left EAR: 0.27, Right EAR: 0.29, Avg EAR: 0.28
Left EAR: 0.28, Right EAR: 0.30, Avg EAR: 0.29
Left EAR: 0.3

Left EAR: 0.15, Right EAR: 0.16, Avg EAR: 0.15
Left EAR: 0.15, Right EAR: 0.17, Avg EAR: 0.16
Left EAR: 0.14, Right EAR: 0.15, Avg EAR: 0.15
Left EAR: 0.17, Right EAR: 0.17, Avg EAR: 0.17
Left EAR: 0.12, Right EAR: 0.11, Avg EAR: 0.12
Left EAR: 0.15, Right EAR: 0.17, Avg EAR: 0.16
Left EAR: 0.14, Right EAR: 0.16, Avg EAR: 0.15
Left EAR: 0.12, Right EAR: 0.13, Avg EAR: 0.12
Left EAR: 0.13, Right EAR: 0.13, Avg EAR: 0.13
Left EAR: 0.15, Right EAR: 0.17, Avg EAR: 0.16
Left EAR: 0.16, Right EAR: 0.17, Avg EAR: 0.16
Left EAR: 0.15, Right EAR: 0.18, Avg EAR: 0.17
Left EAR: 0.17, Right EAR: 0.16, Avg EAR: 0.16
Left EAR: 0.15, Right EAR: 0.16, Avg EAR: 0.16
Left EAR: 0.17, Right EAR: 0.17, Avg EAR: 0.17
Left EAR: 0.19, Right EAR: 0.23, Avg EAR: 0.21
Left EAR: 0.16, Right EAR: 0.17, Avg EAR: 0.17
Left EAR: 0.14, Right EAR: 0.18, Avg EAR: 0.16
Left EAR: 0.18, Right EAR: 0.17, Avg EAR: 0.18
Left EAR: 0.13, Right EAR: 0.13, Avg EAR: 0.13
Left EAR: 0.17, Right EAR: 0.19, Avg EAR: 0.18
Left EAR: 0.1